In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

In [2]:
data=pd.read_csv('imbalancedFinal.csv')


In [3]:
filtered_data4 = data[data['Road user'] == 3]
filtered_data4.shape

(12354, 15)

In [4]:
num_bins = 2

bed_days_min = filtered_data4['Severity'].min()
bed_days_max = filtered_data4['Severity'].max()

bin_width = (bed_days_max - bed_days_min) / num_bins
bins = [bed_days_min + i * bin_width for i in range(num_bins + 1)]

def bin_function(x):
    for i in range(1, len(bins)):
        if x <= bins[i]:
            return i - 1
    return i

filtered_data4['Bed_days_category'] = filtered_data4['Severity'].apply(bin_function)

/var/folders/3l/xh28kn0j7_7d5hh7fgz188940000gn/T/ipykernel_3294/3159902379.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data4['Bed_days_category'] = filtered_data4['Severity'].apply(bin_function)


In [5]:
filtered_data4['Bed_days_category'].value_counts()


0    12262
1       92
Name: Bed_days_category, dtype: int64

In [6]:
from xgboost import XGBClassifier


X = filtered_data4[['Calendar year','Month','Cause of injury','Age group','Sex','Counterparty','ABS remoteness area','Road user']]


y = filtered_data4['Bed_days_category']



X_train , X_test , y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

xgb_model = XGBClassifier(random_state=42)

xgb_model.fit(X_train, y_train)


y_pred_xgb = xgb_model.predict(X_test)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
class_report_xgb = classification_report(y_test, y_pred_xgb)

precision_xgb = precision_score(y_test, y_pred_xgb) 
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
print('Accuracy:', acc_xgb)
print('Confusion Matrix:\n', conf_matrix_xgb)
print('Classification Report:\n', class_report_xgb)
print('Precision:', precision_xgb)
print('Recall:', recall_xgb)
print('F1-score:', f1_xgb)

Accuracy: 0.9962233612085244
Confusion Matrix:
 [[3671    6]
 [   8   22]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3677
           1       0.79      0.73      0.76        30

    accuracy                           1.00      3707
   macro avg       0.89      0.87      0.88      3707
weighted avg       1.00      1.00      1.00      3707

Precision: 0.7857142857142857
Recall: 0.7333333333333333
F1-score: 0.7586206896551724


In [8]:
y_pred_proba_svm = xgb_model.predict_proba(X_train)

thresholds = np.linspace(0, 1, 1000)
f1_scores = []
for threshold in thresholds:
    y_pred_threshold = (y_pred_proba_svm[:, 1] >= threshold).astype(int)
    f1_scores.append(f1_score(y_train, y_pred_threshold))

best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]

print("بهترین آستانه با توجه به F1 Score:", best_threshold)
print("مقدار F1 Score بهترین آستانه:", best_f1_score)

بهترین آستانه با توجه به F1 Score: 0.25725725725725723
مقدار F1 Score بهترین آستانه: 0.9682539682539683


In [9]:
threshold = 0.25725725725725723

probabilities = xgb_model.predict_proba(X_test)

thresholded_predictions = (probabilities[:, 1] >= threshold).astype(int)

acc_thresholded = accuracy_score(y_test, thresholded_predictions)
conf_matrix_thresholded = confusion_matrix(y_test, thresholded_predictions)
class_report_thresholded = classification_report(y_test, thresholded_predictions)

precision_thresholded = precision_score(y_test, thresholded_predictions) 
recall_thresholded = recall_score(y_test, thresholded_predictions)
f1_thresholded = f1_score(y_test, thresholded_predictions)
print('Accuracy:', acc_thresholded)
print('Confusion Matrix:\n', conf_matrix_thresholded)
print('Classification Report:\n', class_report_thresholded)
print('Precision:', precision_thresholded)
print('Recall:', recall_thresholded)
print('F1-score:', f1_thresholded)


Accuracy: 0.9951443215538172
Confusion Matrix:
 [[3665   12]
 [   6   24]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3677
           1       0.67      0.80      0.73        30

    accuracy                           1.00      3707
   macro avg       0.83      0.90      0.86      3707
weighted avg       1.00      1.00      1.00      3707

Precision: 0.6666666666666666
Recall: 0.8
F1-score: 0.7272727272727272


In [7]:
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)

# Evaluate the model
y_pred_lr = lr_model.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)
conf_matrix_lr = confusion_matrix(y_test, y_pred_lr)
class_report_lr = classification_report(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr)
recall_lr = recall_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr)

print('Accuracy:', acc_lr)
print('Confusion Matrix:\n', conf_matrix_lr)
print('Classification Report:\n', class_report_lr)
print('Precision:', precision_lr)
print('Recall:', recall_lr)
print('F1-score:', f1_lr)

Accuracy: 0.9919072025896952
Confusion Matrix:
 [[3677    0]
 [  30    0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      3677
           1       0.00      0.00      0.00        30

    accuracy                           0.99      3707
   macro avg       0.50      0.50      0.50      3707
weighted avg       0.98      0.99      0.99      3707

Precision: 0.0
Recall: 0.0
F1-score: 0.0


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-

In [8]:
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', random_state=42)

# آموزش مدل با داده‌های آموزشی
mlp_model.fit(X_train
              , y_train)

# Evaluate the model
y_pred_mlp = mlp_model.predict(X_test)
acc_mlp = accuracy_score(y_test, y_pred_mlp)
conf_matrix_mlp = confusion_matrix(y_test, y_pred_mlp)
class_report_mlp = classification_report(y_test, y_pred_mlp)
precision_mlp = precision_score(y_test, y_pred_mlp)
recall_mlp = recall_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp)

print('Accuracy:', acc_mlp)
print('Confusion Matrix:\n', conf_matrix_mlp)
print('Classification Report:\n', class_report_mlp)
print('Precision:', precision_mlp)
print('Recall:', recall_mlp)
print('F1-score:', f1_mlp)

Accuracy: 0.9919072025896952
Confusion Matrix:
 [[3677    0]
 [  30    0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      3677
           1       0.00      0.00      0.00        30

    accuracy                           0.99      3707
   macro avg       0.50      0.50      0.50      3707
weighted avg       0.98      0.99      0.99      3707

Precision: 0.0
Recall: 0.0
F1-score: 0.0


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anacon

In [9]:
model = SVC(random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)

acc = accuracy_score(y_test, pred)
conf = confusion_matrix(y_test, pred)
class_report = classification_report(y_test, pred)

precision = precision_score(y_test, pred) 
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)

print('Accuracy:', acc)
print('Confusion Matrix:\n', conf)
print('Classification Report:\n', class_report)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

Accuracy: 0.9919072025896952
Confusion Matrix:
 [[3677    0]
 [  30    0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      3677
           1       0.00      0.00      0.00        30

    accuracy                           0.99      3707
   macro avg       0.50      0.50      0.50      3707
weighted avg       0.98      0.99      0.99      3707

Precision: 0.0
Recall: 0.0
F1-score: 0.0


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anacon

In [10]:
model =DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)



acc = accuracy_score(y_test, pred)
conf = confusion_matrix(y_test, pred)
class_report = classification_report(y_test, pred)

precision = precision_score(y_test, pred) 
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)

print('Accuracy:', acc)
print('Confusion Matrix:\n', conf)
print('Classification Report:\n', class_report)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

Accuracy: 0.9951443215538172
Confusion Matrix:
 [[3664   13]
 [   5   25]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3677
           1       0.66      0.83      0.74        30

    accuracy                           1.00      3707
   macro avg       0.83      0.91      0.87      3707
weighted avg       1.00      1.00      1.00      3707

Precision: 0.6578947368421053
Recall: 0.8333333333333334
F1-score: 0.7352941176470588
